In [15]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"O {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Nov  2 00:24:34 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 32
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00000

diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpq874dxti
max_memory 4000 MB (current use 381 MB)
number electrons alpha = 16  beta = 16
Set gradient conv threshold to 3.16228e-05
init E= -300.573836182351

WARN: alpha nocc = 16  HOMO -0.0811247460588339 >= LUMO -0.0811247460588323


WARN: beta  nocc = 16  HOMO -0.0908913940815762 >= LUMO -0.0908913940815742


WARN: system HOMO -0.0908913940815762 >= system LUMO -0.0908913940815742

cycle= 1 E= -296.965743736783  delta_E= 3.61  |g|= 0.432  |ddm|= 2.88
  alpha nocc = 16  HOMO = -0.219245859288416  LUMO = 0.174608570193703
  beta  nocc = 16  HOMO = -0.219111705597216  LUMO = 0.174804050159882
cycle= 2 E= -297.056250595846  delta_E= -0.0905  |g|= 0.125  |ddm|= 0.794
  alpha nocc = 16  HOMO = -0.208234107338205  LUMO = 0.21164139578728
  beta  nocc = 16  HOMO = -0.208036866918758  LUMO = 0.2120451

-0.19679417445987535

In [16]:
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print(mf.e_tot)
print(mf.e_tot+eccs)
mycc.t1 = (mycc.t1[0]*10,mycc.t1[1]*10)
eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print(mf.e_tot+eccs)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
print(mf.e_tot+eccsd)

-297.0614919796749
-297.0613613403424
-297.0484284650189
-297.2453532788113


In [14]:
def thouless_trans(t1):
    ''' thouless transformation |psi'> = exp(t1)|psi>
        gives the transformed mo_occrep in the 
        original mo basis <psi_p|psi'_i>
        t = t_ia
        t_ia = c_ik c.T_ka
        c_ik = <psi_i|psi'_k>
    '''
    q, r = jnp.linalg.qr(t1,mode='complete')
    u_ji = q
    u_ai = r.T
    u_occ = jnp.vstack((u_ji,u_ai))
    mo_t, r = jnp.linalg.qr(u_occ,mode='complete')
    # sgn = np.sign(r.diagonal())
    # sgn = np.sign((mo_t).diagonal())
    # # choose the mo_t s.t. has 
    # # positive olp with the original mo
    # mo_t = np.einsum("ij,j->ij", mo_t, sgn)
    return mo_t

In [17]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'uccsd_pt2_ad',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import config
from ad_afqmc.prop_unrestricted import prop_unrestricted
import time
from jax import random
# from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = [10*mycc.t1[0],10*mycc.t1[1]]
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
prop_unrestricted.prep_afqmc(mycc,options,chol_cut=1e-6)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (prop_unrestricted._prep_afqmc(options))


#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (16, 16)
# Number of basis functions: 20
# Number of Cholesky vectors: 104
#
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 20
# nelec: (16, 16)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: uhf
# trial: uccsd_pt2_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [18]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
# et1 = jnp.real(trial._calc_energy(
#     walker_up, walker_dn, ham_data, wave_data
#     ))
# print('exact T1 transformed init walker energy: ', et1)
print(trial._calc_energy(walker_up,walker_dn,ham_data,wave_data))
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print('ccs energy: ', mf.e_tot+eccsd)
print('ccsd energy: ', mycc.e_tot)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -297.06147809974493
mf enegry:  -297.0614919796749
(-297.06147809974493+0j)
ccs energy:  -297.0484284650189
ccsd energy:  -297.25828615413474


In [19]:
t2aa = jnp.array(mycc.t2[0])
t2ab = jnp.array(mycc.t2[1])
t2bb = jnp.array(mycc.t2[2])
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
wave_data["t2aa"] = t2aa.transpose(0, 2, 1, 3)
wave_data["t2ab"] = t2ab.transpose(0, 2, 1, 3)
wave_data["t2bb"] = t2bb.transpose(0, 2, 1, 3)
wave_data["t1a"] = mycc.t1[0]
wave_data["t1b"] = mycc.t1[1]

In [86]:
noccA = trial.nelec[0]
noccB = trial.nelec[1]
wave_data['mo_ta'] = np.eye(trial.norb)[:,:noccA]
wave_data['mo_tb'] = np.eye(trial.norb)[:,:noccB]
wave_data["t2aa"] = wave_data["ci2AA"]
wave_data["t2ab"] = wave_data["ci2AB"]
wave_data["t2bb"] = wave_data["ci2BB"]

In [83]:
from jax import lax
@partial(jit, static_argnums=0)
def _calc_ucisd_energy(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    nocc_a, ci1_a, ci2_aa = trial.nelec[0], wave_data["ci1A"], wave_data["ci2AA"]
    nocc_b, ci1_b, ci2_bb = trial.nelec[1], wave_data["ci1B"], wave_data["ci2BB"]
    ci2_ab = wave_data["ci2AB"]
    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:nocc_a, :]))).T
    green_b = (walker_dn.dot(jnp.linalg.inv(walker_dn[:nocc_b, :]))).T
    green_occ_a = green_a[:, nocc_a:].copy()
    green_occ_b = green_b[:, nocc_b:].copy()
    greenp_a = jnp.vstack((green_occ_a, -jnp.eye(trial.norb - nocc_a)))
    greenp_b = jnp.vstack((green_occ_b, -jnp.eye(trial.norb - nocc_b)))

    chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
    rot_chol_a = chol_a[:, :nocc_a, :]
    rot_chol_b = chol_b[:, :nocc_b, :]
    h1_a = ham_data["h1"][0]
    h1_b = ham_data["h1"][1]
    hg_a = jnp.einsum("pj,pj->", h1_a[:nocc_a, :], green_a)
    hg_b = jnp.einsum("pj,pj->", h1_b[:nocc_b, :], green_b)
    hg = hg_a + hg_b

    # 0 body energy
    h0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = hg # <HF|h1|walker>/<HF|walker>

    # single excitations
    # ci1g_a = jnp.einsum("pt,pt->", ci1_a, green_occ_a, optimize="optimal")
    # ci1g_b = jnp.einsum("pt,pt->", ci1_b, green_occ_b, optimize="optimal")
    # ci1g = ci1g_a + ci1g_b
    # e1_1_1 = ci1g * hg
    # gpci1_a = greenp_a @ ci1_a.T
    # gpci1_b = greenp_b @ ci1_b.T
    # ci1_green_a = gpci1_a @ green_a
    # ci1_green_b = gpci1_b @ green_b
    # e1_1_2 = -(
    #     jnp.einsum("ij,ij->", h1_a, ci1_green_a, optimize="optimal")
    #     + jnp.einsum("ij,ij->", h1_b, ci1_green_b, optimize="optimal")
    # )
    # e1_1 = e1_1_1 + e1_1_2

    # double excitations
    ci2g_a = jnp.einsum("ptqu,pt->qu", ci2_aa, green_occ_a) / 4
    ci2g_b = jnp.einsum("ptqu,pt->qu", ci2_bb, green_occ_b) / 4
    ci2g_ab_a = jnp.einsum("ptqu,qu->pt", ci2_ab, green_occ_b)
    ci2g_ab_b = jnp.einsum("ptqu,pt->qu", ci2_ab, green_occ_a)
    gci2g_a = jnp.einsum("qu,qu->", ci2g_a, green_occ_a, optimize="optimal")
    gci2g_b = jnp.einsum("qu,qu->", ci2g_b, green_occ_b, optimize="optimal")
    gci2g_ab = jnp.einsum("pt,pt->", ci2g_ab_a, green_occ_a, optimize="optimal")
    gci2g = 2 * (gci2g_a + gci2g_b) + gci2g_ab
    e1_2_1 = hg * gci2g
    ci2_green_a = (greenp_a @ ci2g_a.T) @ green_a
    ci2_green_ab_a = (greenp_a @ ci2g_ab_a.T) @ green_a
    ci2_green_b = (greenp_b @ ci2g_b.T) @ green_b
    ci2_green_ab_b = (greenp_b @ ci2g_ab_b.T) @ green_b
    e1_2_2_a = -jnp.einsum(
        "ij,ij->", h1_a, 4 * ci2_green_a + ci2_green_ab_a, optimize="optimal"
    )
    e1_2_2_b = -jnp.einsum(
        "ij,ij->", h1_b, 4 * ci2_green_b + ci2_green_ab_b, optimize="optimal"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2

    e1_1 = 0
    e1 = e1_0 + e1_1 + e1_2

    # two body energy
    # ref
    lg_a = jnp.einsum("gpj,pj->g", rot_chol_a, green_a, optimize="optimal")
    lg_b = jnp.einsum("gpj,pj->g", rot_chol_b, green_b, optimize="optimal")
    e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
    lg1_a = jnp.einsum("gpj,qj->gpq", rot_chol_a, green_a, optimize="optimal")
    lg1_b = jnp.einsum("gpj,qj->gpq", rot_chol_b, green_b, optimize="optimal")
    e2_0_2 = (
        -(
            jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
            + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
        )
        / 2.0
    )
    e2_0 = e2_0_1 + e2_0_2 # <HF|h2|walker>/<HF|walker>

    # single excitations
    # e2_1_1 = e2_0 * ci1g
    # lci1g_a = jnp.einsum("gij,ij->g", chol_a, ci1_green_a, optimize="optimal")
    # lci1g_b = jnp.einsum("gij,ij->g", chol_b, ci1_green_b, optimize="optimal")
    # e2_1_2 = -((lci1g_a + lci1g_b) @ (lg_a + lg_b))
    # ci1g1_a = ci1_a @ green_occ_a.T
    # ci1g1_b = ci1_b @ green_occ_b.T
    # e2_1_3_1 = jnp.einsum(
    #     "gpq,gqr,rp->", lg1_a, lg1_a, ci1g1_a, optimize="optimal"
    # ) + jnp.einsum("gpq,gqr,rp->", lg1_b, lg1_b, ci1g1_b, optimize="optimal")
    # lci1g_a = jnp.einsum(
    #     "gip,qi->gpq", ham_data["lci1_a"], green_a, optimize="optimal"
    # )
    # lci1g_b = jnp.einsum(
    #     "gip,qi->gpq", ham_data["lci1_b"], green_b, optimize="optimal"
    # )
    # e2_1_3_2 = -jnp.einsum(
    #     "gpq,gqp->", lci1g_a, lg1_a, optimize="optimal"
    # ) - jnp.einsum("gpq,gqp->", lci1g_b, lg1_b, optimize="optimal")
    # e2_1_3 = e2_1_3_1 + e2_1_3_2
    # e2_1 = e2_1_1 + e2_1_2 + e2_1_3

    # double excitations
    e2_2_1 = e2_0 * gci2g
    lci2g_a = jnp.einsum(
        "gij,ij->g",
        chol_a,
        8 * ci2_green_a + 2 * ci2_green_ab_a,
        optimize="optimal",
    )
    lci2g_b = jnp.einsum(
        "gij,ij->g",
        chol_b,
        8 * ci2_green_b + 2 * ci2_green_ab_b,
        optimize="optimal",
    )
    e2_2_2_1 = -((lci2g_a + lci2g_b) @ (lg_a + lg_b)) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, rot_chol_a_i, chol_b_i, rot_chol_b_i = x
        gl_a_i = jnp.einsum("pj,ji->pi", green_a, chol_a_i, optimize="optimal")
        gl_b_i = jnp.einsum("pj,ji->pi", green_b, chol_b_i, optimize="optimal")
        lci2_green_a_i = jnp.einsum(
            "pi,ji->pj",
            rot_chol_a_i,
            8 * ci2_green_a + 2 * ci2_green_ab_a,
            optimize="optimal",
        )
        lci2_green_b_i = jnp.einsum(
            "pi,ji->pj",
            rot_chol_b_i,
            8 * ci2_green_b + 2 * ci2_green_ab_b,
            optimize="optimal",
        )
        carry[0] += 0.5 * (
            jnp.einsum("pi,pi->", gl_a_i, lci2_green_a_i, optimize="optimal")
            + jnp.einsum("pi,pi->", gl_b_i, lci2_green_b_i, optimize="optimal")
        )
        glgp_a_i = jnp.einsum(
            "pi,it->pt", gl_a_i, greenp_a, optimize="optimal"
        ).astype(jnp.complex64)
        glgp_b_i = jnp.einsum(
            "pi,it->pt", gl_b_i, greenp_b, optimize="optimal"
        ).astype(jnp.complex64)
        l2ci2_a = 0.5 * jnp.einsum(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_a_i,
            ci2_aa.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_b = 0.5 * jnp.einsum(
            "pt,qu,ptqu->",
            glgp_b_i,
            glgp_b_i,
            ci2_bb.astype(jnp.float32),
            optimize="optimal",
        )
        l2ci2_ab = jnp.einsum(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_b_i,
            ci2_ab.astype(jnp.float32),
            optimize="optimal",
        )
        carry[1] += l2ci2_a + l2ci2_b + l2ci2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(
        scanned_fun, [0.0, 0.0], (chol_a, rot_chol_a, chol_b, rot_chol_b)
    )
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    e2_1 = 0
    e2 = e2_0 + e2_1 + e2_2

    # overlap
    # overlap_1 = ci1g
    overlap_2 = gci2g
    overlap = 1.0 + overlap_2
    t =  gci2g
    E0 = e1_0 + e2_0
    E1 = e1_2 + e2_2
    return t, E0, E1, h0 + (e1 + e2) / overlap

In [11]:
from jax import lax
@partial(jit, static_argnums=0)
def _calc_energy_pt2(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    nocc_a, t2_aa = trial.nelec[0], wave_data["t2aa"]
    nocc_b, t2_bb = trial.nelec[1], wave_data["t2bb"]
    t2_ab = wave_data["t2ab"]
    mo_a, mo_b = wave_data['mo_ta'], wave_data['mo_tb']
    chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
    # rot_chol_a = ham_data["rot_chol"][0]
    # rot_chol_b = ham_data["rot_chol"][1]
    h1_a = ham_data["h1"][0]
    h1_b = ham_data["h1"][1]

    # full green's function G_pq
    green_a = (walker_up @ (jnp.linalg.inv(mo_a.T @ walker_up)) @ mo_a.T).T
    green_b = (walker_dn @ (jnp.linalg.inv(mo_b.T @ walker_dn)) @ mo_b.T).T
    # green_ov_a = green_a[:nocc_a, nocc_a:].copy()
    # green_ov_b = green_b[:nocc_b, nocc_b:].copy()
    # greenp_a = jnp.vstack((green_ov_a, -jnp.eye(trial.norb - nocc_a)))
    # greenp_b = jnp.vstack((green_ov_b, -jnp.eye(trial.norb - nocc_b)))
    # half_Gp_iq = (phi_qk inv(psi^dagger @ phi)_ki - trial_mo_qi).T
    greenp_a = (green_a - jnp.eye(trial.norb))[:,nocc_a:]
    greenp_b = (green_b - jnp.eye(trial.norb))[:,nocc_b:]

    hg_a = jnp.einsum("pq,pq->", h1_a, green_a) # rot_h_iq half_G_iq alpha
    hg_b = jnp.einsum("pq,pq->", h1_b, green_b) # rot_h_iq half_G_iq beta
    hg = hg_a + hg_b # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>

    # 0 body energy
    h0 = ham_data["h0"]

    # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>
    # one body energy
    e1_0 = hg

    # <exp(T1)HF|h2|walker>/<exp(T1)HF|walker>
    # two body energy
    lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
    lg1_a = jnp.einsum("gpj,qj->gpq", chol_a, green_a, optimize="optimal")
    lg1_b = jnp.einsum("gpj,qj->gpq", chol_b, green_b, optimize="optimal")
    e2_0_2 = (
        -(
            jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
            + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
        )
        / 2.0
    )
    e2_0 = e2_0_1 + e2_0_2

    # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>
    # double excitations
    t2g_a = jnp.einsum("iajb,ia->jb", t2_aa, green_a[:nocc_a,nocc_a:]) / 4
    t2g_b = jnp.einsum("iajb,ia->jb", t2_bb, green_b[:nocc_b,nocc_b:]) / 4
    t2g_ab_a = jnp.einsum("iajb,jb->ia", t2_ab, green_b[:nocc_b,nocc_b:])
    t2g_ab_b = jnp.einsum("iajb,ia->jb", t2_ab, green_a[:nocc_a,nocc_a:])
    # t_iajb (G_ia G_jb - G_ib G_ja)
    gt2g_a = jnp.einsum("jb,jb->", t2g_a, green_a[:nocc_a,nocc_a:], 
                        optimize="optimal")
    gt2g_b = jnp.einsum("jb,jb->", t2g_b, green_b[:nocc_b,nocc_b:], 
                        optimize="optimal")
    gt2g_ab = jnp.einsum("ia,ia->", t2g_ab_a, green_a[:nocc_a,nocc_a:], 
                         optimize="optimal")
    gt2g = 2 * (gt2g_a + gt2g_b) + gt2g_ab # <exp(T1)HF|T2|walker>/<exp(T1)HF|walker>

    e1_2_1 = hg * gt2g
    
    t2_green_a = (greenp_a @ t2g_a.T) @ green_a[:nocc_a,:] # Gp_pb t_iajb G_ia G_jq
    t2_green_ab_a = (greenp_a @ t2g_ab_a.T) @ green_a[:nocc_a,:]
    t2_green_b = (greenp_b @ t2g_b.T) @ green_b[:nocc_b,:]
    t2_green_ab_b = (greenp_b @ t2g_ab_b.T) @ green_b[:nocc_b,:]
    e1_2_2_a = -jnp.einsum(
        "pq,pq->", h1_a, 4 * t2_green_a + t2_green_ab_a, optimize="optimal"
    )
    e1_2_2_b = -jnp.einsum(
        "pq,pq->", h1_b, 4 * t2_green_b + t2_green_ab_b, optimize="optimal"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2  # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>

    # e1 = e1_0 + e1_1 + e1_2

    # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>
    # double excitations
    e2_2_1 = e2_0 * gt2g
    lt2g_a = jnp.einsum("gpq,pq->g",
                        chol_a, 8 * t2_green_a + 2 * t2_green_ab_a,
                        optimize="optimal")
    lt2g_b = jnp.einsum("gpq,pq->g",
        chol_b, 8 * t2_green_b + 2 * t2_green_ab_b,
        optimize="optimal")
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ (lg_a + lg_b)) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, chol_b_i = x
        gl_a_i = jnp.einsum("pr,rq->pq", green_a, chol_a_i,
                            optimize="optimal")
        gl_b_i = jnp.einsum("pr,rq->pq", green_b, chol_b_i,
                            optimize="optimal")
        lt2_green_a_i = jnp.einsum(
            "pr,qr->pq", chol_a_i, 8 * t2_green_a + 2 * t2_green_ab_a,
            optimize="optimal")
        lt2_green_b_i = jnp.einsum(
            "pr,qr->pq", chol_b_i, 8 * t2_green_b + 2 * t2_green_ab_b,
            optimize="optimal")
        carry[0] += 0.5 * (
            jnp.einsum("pq,pq->", gl_a_i, lt2_green_a_i, optimize="optimal")
            + jnp.einsum("pq,pq->", gl_b_i, lt2_green_b_i, optimize="optimal")
        )
        glgp_a_i = jnp.einsum(
            "iq,qa->ia", gl_a_i[:nocc_a,:], greenp_a, optimize="optimal"
        ).astype(jnp.complex64)
        glgp_b_i = jnp.einsum(
            "iq,qa->ia", gl_b_i[:nocc_b,:], greenp_b, optimize="optimal"
        ).astype(jnp.complex64)
        l2t2_a = 0.5 * jnp.einsum(
            "ia,jb,iajb->",glgp_a_i,glgp_a_i,t2_aa.astype(jnp.float32),
            optimize="optimal")
        l2t2_b = 0.5 * jnp.einsum(
            "ia,jb,iajb->",glgp_b_i,glgp_b_i,t2_bb.astype(jnp.float32),
            optimize="optimal")
        l2t2_ab = jnp.einsum(
            "ia,jb,iajb->",glgp_a_i,glgp_b_i,t2_ab.astype(jnp.float32),
            optimize="optimal")
        carry[1] += l2t2_a + l2t2_b + l2t2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol_a, chol_b))
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>

    # e2 = e2_0 + e2_1 + e2_2
    o0 = jnp.linalg.det(walker_up[:nocc_a,:nocc_a]
        ) * jnp.linalg.det(walker_dn[:nocc_b,:nocc_b])
    # <exp(T1)HF|walker>/<HF|walker>
    t1 = jnp.linalg.det(wave_data["mo_ta"].T.conj() @ walker_up
        ) * jnp.linalg.det(wave_data["mo_tb"].T.conj() @ walker_dn) / o0
    t2 = gt2g * t1 # <exp(T1)HF|T2|walker>/<HF|walker>
    e0 = (e1_0 + e2_0) * t1 # <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = (e1_2 + e2_2) * t1 # <exp(T1)HF|T2 (h1+h2)|walker>/<HF|walker>

    # overlap
    # overlap_1 = t1g
    # overlap_2 = gt2g
    # overlap = 1.0 + overlap_1 + overlap_2
    # return (e1 + e2) / overlap + e0
    return t1, t2, e0, e1, e1_2, e2_2

In [20]:
# mycc.t1 = (mycc.t1[0]*0, mycc.t1[1]*0)
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1,(0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
eccs = mf.e_tot + eccs
print('ccs energy: ', eccs)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
eccsd = mf.e_tot + eccsd
print('ccsd energy: ', eccsd)

ccs energy:  -297.0484284650189
ccsd energy:  -297.2453532788113


In [21]:
h0 = ham_data['h0']
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]
import time
start = time.perf_counter()
t1,t2,e0,e1,e1_2,e2_2 = _calc_energy_pt2(trial, walker_up, walker_dn, ham_data, wave_data)
end = time.perf_counter()
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 )
print(h0 + 1/t1 * e0 - eccs)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

(0.7266550759350222+0j) 0j (-316.61457515077444+0j) (-0.14309632297264166+0j)
(-297.04841416913587+0j)
(1.4295883033810242e-05+0j)
(-297.24533885556775+0j)
(1.4423243555938825e-05+0j)
1.269128300016746


In [94]:
t,e0,e1,e_tot = _calc_ucisd_energy(trial, walker_up, walker_dn, ham_data, wave_data)
print(t,e0,e1)
print(e_tot)

0j (-4.25535904779058+0j) (-0.07659751176834106+0j)
(-2.1652898928922544+0j)


In [43]:
print(e1_2,e2_2)

(-0+0j) (0.04213117063045502+0j)


In [71]:
start = time.perf_counter()
t1,t2,e0,e1 = trial._calc_energy_pt(walker_up, walker_dn, ham_data, wave_data)
end = time.perf_counter()
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 )
print(h0 + 1/t1 * e0 - eccs)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

1.0 0.0 -4.255359056422076 -0.07659750927737548
-2.0886923897554097
-7.807689605954238e-09
-2.165289899032785
-9.119868860096858e-09
2.1349694000091404
